In [1]:
import os
from typing import Tuple, Callable, Dict, Optional, List

import numpy as np
import pandas as pd
import scipy.sparse as sp
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from Evaluation.Evaluator import EvaluatorHoldout
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
from Recommenders.SLIM.Cython.SLIM_BPR_Cython import SLIM_BPR_Cython
from HyperparameterTuning.SearchBayesianSkopt import SearchBayesianSkopt
from HyperparameterTuning.SearchAbstractClass import SearchInputRecommenderArgs
from Recommenders.DataIO import DataIO
from skopt.space import Real, Integer, Categorical
from Recommenders.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
from Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender
from Recommenders.SLIM.Cython.SLIM_BPR_Cython import SLIM_BPR_Cython
from Recommenders.SLIM.SLIMElasticNetRecommender import SLIMElasticNetRecommender,MultiThreadSLIM_SLIMElasticNetRecommender
from Recommenders.SLIM.SLIMElasticNetRecommender import MultiThreadSLIM_SLIM_S_ElasticNetRecommender_Multiple_ICM_stacked
np.random.seed(2498)


columns_name = ['user_id','item_id','impression_list','data']

dataset_l = pd.read_csv('/Users/loren/Documents/GitHub/RecSys/dati/data_ICM_length.csv', sep=',')
dataset_t = pd.read_csv('/Users/loren/Documents/GitHub/RecSys/dati/data_ICM_type.csv', sep=',')
dataset = pd.read_csv('/Users/loren/Documents/GitHub/RecSys/dati/interactions_and_impressions.csv', sep=',')
test_users = pd.read_csv('/Users/loren/Documents/GitHub/RecSys/dati/data_target_users_test.csv',sep = ',')
dataset.columns = columns_name

ImportError: cannot import name 'MultiThreadSLIM_SLIMElasticNetRecommender' from 'Recommenders.SLIM.SLIMElasticNetRecommender' (c:\Users\loren\Documents\GitHub\RecSys\Recommenders\SLIM\SLIMElasticNetRecommender.py)

In [ ]:
len(dataset.item_id.unique()),dataset.item_id.unique().min(),dataset.item_id.unique().max()

(24507, 0, 24506)

In [ ]:
from Data_manager.Build_Matrix.Data_import import build_ICM_type,build_ICM_lengh_kmeans_3,build_URM_ICM_onlyURM_item
import scipy.sparse as sps
URM_ALL,ICM=build_URM_ICM_onlyURM_item(dataset, dataset_t, dataset_l)
ICM_TYPE   =build_ICM_type(dataset_t)
ICM_LENGHT =build_ICM_lengh_kmeans_3(dataset_l,n_clusters = 5)
ICM_genre_subgenre = sps.hstack((ICM_TYPE, ICM_LENGHT));

c:\Users\loren\anaconda3\envs\RecSysFramework\lib\site-packages\pandas\core\generic.py:5494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [ ]:
URM_train_validation, URM_test = split_train_in_two_percentage_global_sample(URM_ALL, train_percentage = 0.80)
URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_train_validation, train_percentage = 0.80)

evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])
evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10])

EvaluatorHoldout: Ignoring 697 ( 1.7%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 308 ( 0.7%) Users that have less than 1 test interactions


In [ ]:
param_1 = {'l1_ratio': 5.371794099882435e-05, 'topK': 1651, 'alpha': 0.008001600311161093, 'workers': 2, 'mw': 1}

In [ ]:
p=MultiThreadSLIM_SLIM_S_ElasticNetRecommender_Multiple_ICM_stacked(URM_train,ICM_TYPE,ICM_LENGHT)

In [ ]:
import os

output_folder_path = "result_MultiThreadSLIM_SLIM_S_ElasticNetRecommender_Multiple_ICM_stacked"

# If directory does not exist, create
if not os.path.exists(output_folder_path):
    os.makedirs(output_folder_path)

n_cases = 500
n_random_starts = int(n_cases*0.3)
metric_to_optimize = "MAP"   
cutoff_to_optimize = 10

hyperparameters_range_dictionary = {
        "l1_ratio": Real(low = 1e-5, high = 0.1, prior = 'log-uniform'),
        "topK": Integer(200, 6000),
        "alpha": Real(low = 1e-5, high = 0.1, prior = 'uniform'),
        "workers": Categorical([5]),
        "mw": Integer(0, 10),
        "mw_1": Integer(0, 10)
    }

earlystopping_keywargs = {"validation_every_n": 5,
                              "stop_on_validation": True,
                              "evaluator_object": URM_validation,
                              "lower_validations_allowed": 5,
                              "validation_metric": metric_to_optimize,
                              }
recommender_input_args = SearchInputRecommenderArgs(
                CONSTRUCTOR_POSITIONAL_ARGS = [URM_train,ICM_TYPE,ICM_LENGHT],
                CONSTRUCTOR_KEYWORD_ARGS = {},
                FIT_POSITIONAL_ARGS = [],
                FIT_KEYWORD_ARGS = {},
                EARLYSTOPPING_KEYWORD_ARGS = {},
            )

recommender_class = MultiThreadSLIM_SLIM_S_ElasticNetRecommender_Multiple_ICM_stacked


hyperparameterSearch = SearchBayesianSkopt(recommender_class,
                                        evaluator_validation=evaluator_validation,
                                        evaluator_test=evaluator_test)

In [ ]:
hyperparameterSearch.search(recommender_input_args,
                    hyperparameter_search_space = hyperparameters_range_dictionary,
                    n_cases = n_cases,
                    n_random_starts = n_random_starts,
                    save_model = "last",
                    output_folder_path = output_folder_path, # Where to save the results
                    output_file_name_root = recommender_class.RECOMMENDER_NAME, # How to call the files
                    metric_to_optimize = metric_to_optimize,
                    cutoff_to_optimize = cutoff_to_optimize,
                    resume_from_saved = True,
                    recommender_input_args_last_test=recommender_input_args
                    )

SearchBayesianSkopt: Resuming 'SLIMElasticNetRecommender'... Loaded 46 configurations.
Iteration No: 1 started. Evaluating function at random point.
Iteration No: 1 ended. Evaluation done at random point.
Time taken: 0.0000
Function value obtained: 65504.0000
Current minimum: -0.0223
Iteration No: 2 started. Evaluating function at random point.
SearchBayesianSkopt: Testing config: {'l1_ratio': 0.03446168602520511, 'topK': 734, 'alpha': 0.02772812423473481, 'workers': 2, 'mw': 3, 'mw_1': 7}


100%|█████████▉| 24504/24507 [08:06<00:00, 50.32it/s]


EvaluatorHoldout: Processed 40932 (100.0%) in 25.36 sec. Users per second: 1614
SearchBayesianSkopt: Config 46 is suboptimal. Config: {'l1_ratio': 0.03446168602520511, 'topK': 734, 'alpha': 0.02772812423473481, 'workers': 2, 'mw': 3, 'mw_1': 7} - results: PRECISION: 0.0380729, PRECISION_RECALL_MIN_DEN: 0.0614434, RECALL: 0.0570328, MAP: 0.0173726, MAP_MIN_DEN: 0.0279661, MRR: 0.1283039, NDCG: 0.0591369, F1: 0.0456630, HIT_RATE: 0.2777778, ARHR_ALL_HITS: 0.1486324, NOVELTY: 0.0042428, AVERAGE_POPULARITY: 0.2701740, DIVERSITY_MEAN_INTER_LIST: 0.8639621, DIVERSITY_HERFINDAHL: 0.9863941, COVERAGE_ITEM: 0.0208512, COVERAGE_ITEM_HIT: 0.0142408, ITEMS_IN_GT: 0.9953891, COVERAGE_USER: 0.9832569, COVERAGE_USER_HIT: 0.2731269, USERS_IN_GT: 0.9832569, DIVERSITY_GINI: 0.0044729, SHANNON_ENTROPY: 7.0805471, RATIO_DIVERSITY_HERFINDAHL: 0.9866355, RATIO_DIVERSITY_GINI: 0.0077776, RATIO_SHANNON_ENTROPY: 0.5151033, RATIO_AVERAGE_POPULARITY: 5.1931298, RATIO_NOVELTY: 0.2646536, 



Iteration No: 2 ended. Evaluation done at random point.
Time taken: 507.4379
Function value obtained: -0.0174
Current minimum: -0.0223
Iteration No: 3 started. Evaluating function at random point.
SearchBayesianSkopt: Testing config: {'l1_ratio': 4.549602639373225e-05, 'topK': 5182, 'alpha': 0.07250165439519018, 'workers': 2, 'mw': 9, 'mw_1': 1}


EvaluatorHoldout: Processed 40932 (100.0%) in 54.87 sec. Users per second: 746
SearchBayesianSkopt: Config 47 is suboptimal. Config: {'l1_ratio': 4.549602639373225e-05, 'topK': 5182, 'alpha': 0.07250165439519018, 'workers': 2, 'mw': 9, 'mw_1': 1} - results: PRECISION: 0.0414761, PRECISION_RECALL_MIN_DEN: 0.0675922, RECALL: 0.0628620, MAP: 0.0184269, MAP_MIN_DEN: 0.0297649, MRR: 0.1348706, NDCG: 0.0635395, F1: 0.0499773, HIT_RATE: 0.3005961, ARHR_ALL_HITS: 0.1569539, NOVELTY: 0.0040281, AVERAGE_POPULARITY: 0.3015724, DIVERSITY_MEAN_INTER_LIST: 0.8412434, DIVERSITY_HERFINDAHL: 0.9841223, COVERAGE_ITEM: 0.1594646, COVERAGE_ITEM_HIT: 0.0234627, ITEMS_IN_GT: 0.9953891, COVERAGE_USER: 0.9832569, COVERAGE_USER_HIT: 0.2955632, USERS_IN_GT: 0.9832569, DIVERSITY_GINI: 0.0080774, SHANNON_ENTROPY: 7.3435842, RATIO_DIVERSITY_HERFINDAHL: 0.9843631, RATIO_DIVERSITY_GINI: 0.0140453, RATIO_SHANNON_ENTROPY: 0.5342390, RATIO_AVERAGE_POPULARITY: 5.7965865, RATIO_NOVELTY: 0.2512632, 



...

100%|█████████▉| 24504/24507 [47:20<00:00,  8.63it/s]


EvaluatorHoldout: Processed 40932 (100.0%) in 54.23 sec. Users per second: 755
SearchBayesianSkopt: Config 131 is suboptimal. Config: {'l1_ratio': 5.68415493413365e-05, 'topK': 2938, 'alpha': 0.07112320248965541, 'workers': 2, 'mw': 6, 'mw_1': 5} - results: PRECISION: 0.0414541, PRECISION_RECALL_MIN_DEN: 0.0674936, RECALL: 0.0627599, MAP: 0.0184553, MAP_MIN_DEN: 0.0298039, MRR: 0.1351093, NDCG: 0.0635679, F1: 0.0499291, HIT_RATE: 0.3004495, ARHR_ALL_HITS: 0.1572070, NOVELTY: 0.0040460, AVERAGE_POPULARITY: 0.2993318, DIVERSITY_MEAN_INTER_LIST: 0.8440596, DIVERSITY_HERFINDAHL: 0.9844039, COVERAGE_ITEM: 0.1819072, COVERAGE_ITEM_HIT: 0.0235443, ITEMS_IN_GT: 0.9953891, COVERAGE_USER: 0.9832569, COVERAGE_USER_HIT: 0.2954191, USERS_IN_GT: 0.9832569, DIVERSITY_GINI: 0.0087854, SHANNON_ENTROPY: 7.3893337, RATIO_DIVERSITY_HERFINDAHL: 0.9846384, RATIO_DIVERSITY_GINI: 0.0151433, RATIO_SHANNON_ENTROPY: 0.5368115, RATIO_AVERAGE_POPULARITY: 5.7403559, RATIO_NOVELTY: 0.2479667, 

Iteration No: 87 ende

EvaluatorHoldout: Processed 40932 (100.0%) in 31.94 sec. Users per second: 1281
SearchBayesianSkopt: Config 132 is suboptimal. Config: {'l1_ratio': 0.06276729322610702, 'topK': 1057, 'alpha': 0.0626649616144965, 'workers': 2, 'mw': 1, 'mw_1': 2} - results: PRECISION: 0.0246653, PRECISION_RECALL_MIN_DEN: 0.0386684, RECALL: 0.0355820, MAP: 0.0115405, MAP_MIN_DEN: 0.0183761, MRR: 0.0905244, NDCG: 0.0390668, F1: 0.0291346, HIT_RATE: 0.1892651, ARHR_ALL_HITS: 0.1018000, NOVELTY: 0.0046823, AVERAGE_POPULARITY: 0.2458429, DIVERSITY_MEAN_INTER_LIST: 0.7572761, DIVERSITY_HERFINDAHL: 0.9757258, COVERAGE_ITEM: 0.0058351, COVERAGE_ITEM_HIT: 0.0034276, ITEMS_IN_GT: 0.9953891, COVERAGE_USER: 0.9832569, COVERAGE_USER_HIT: 0.1860962, USERS_IN_GT: 0.9832569, DIVERSITY_GINI: 0.0016099, SHANNON_ENTROPY: 5.6873382, RATIO_DIVERSITY_HERFINDAHL: 0.9760193, RATIO_DIVERSITY_GINI: 0.0030371, RATIO_SHANNON_ENTROPY: 0.4188236, RATIO_AVERAGE_POPULARITY: 4.8028309, RATIO_NOVELTY: 0.3339208, 



Iteration No: 88 ended. Evaluation done at random point.
Time taken: 410.8312
Function value obtained: -0.0115
Current minimum: -0.0223
Iteration No: 89 started. Evaluating function at random point.
SearchBayesianSkopt: Testing config: {'l1_ratio': 5.894553552288058e-05, 'topK': 1795, 'alpha': 0.04257439514218121, 'workers': 2, 'mw': 6, 'mw_1': 5}


EvaluatorHoldout: Processed 40932 (100.0%) in 50.48 sec. Users per second: 811
SearchBayesianSkopt: Config 133 is suboptimal. Config: {'l1_ratio': 5.894553552288058e-05, 'topK': 1795, 'alpha': 0.04257439514218121, 'workers': 2, 'mw': 6, 'mw_1': 5} - results: PRECISION: 0.0431374, PRECISION_RECALL_MIN_DEN: 0.0698547, RECALL: 0.0648779, MAP: 0.0194200, MAP_MIN_DEN: 0.0312118, MRR: 0.1403920, NDCG: 0.0661660, F1: 0.0518198, HIT_RATE: 0.3083407, ARHR_ALL_HITS: 0.1643481, NOVELTY: 0.0041021, AVERAGE_POPULARITY: 0.2790621, DIVERSITY_MEAN_INTER_LIST: 0.8703986, DIVERSITY_HERFINDAHL: 0.9870377, COVERAGE_ITEM: 0.1844779, COVERAGE_ITEM_HIT: 0.0264822, ITEMS_IN_GT: 0.9953891, COVERAGE_USER: 0.9832569, COVERAGE_USER_HIT: 0.3031781, USERS_IN_GT: 0.9832569, DIVERSITY_GINI: 0.0100939, SHANNON_ENTROPY: 7.6774815, RATIO_DIVERSITY_HERFINDAHL: 0.9872729, RATIO_DIVERSITY_GINI: 0.0173988, RATIO_SHANNON_ENTROPY: 0.5577445, RATIO_AVERAGE_POPULARITY: 5.3516387, RATIO_NOVELTY: 0.2514031, 

Iteration No: 89 end

In [ ]:
from Recommenders.DataIO import DataIO
from Recommenders.SLIM.SLIMElasticNetRecommender import SLIMElasticNetRecommender
import pandas as pd
recommender_class = SLIMElasticNetRecommender

output_folder_path = "result_experiments"
data_loader = DataIO(folder_path = output_folder_path)
search_metadata = data_loader.load_data(recommender_class.RECOMMENDER_NAME + "_metadata.zip")
time_df = search_metadata["time_df"]

print('mean-time:')
print(time_df['train'].mean())


result_on_test_df = search_metadata["result_on_test_df"]
result_best_on_test = search_metadata["result_on_last"]
print('BEST_HYPERPARAMETERS: ')
print(search_metadata["hyperparameters_best"])
print(pd.DataFrame(result_on_test_df))
print(result_best_on_test)

mean-time:
392.422853257921
BEST_HYPERPARAMETERS: 
{'topK': 668, 'l1_ratio': 0.06262713223835095, 'alpha': 0.001}
          PRECISION PRECISION_RECALL_MIN_DEN    RECALL       MAP MAP_MIN_DEN  \
   cutoff                                                                      
0  10      0.048502                 0.067374  0.059069  0.022471    0.030846   
1  10           NaN                      NaN       NaN       NaN         NaN   
2  10           NaN                      NaN       NaN       NaN         NaN   
3  10           NaN                      NaN       NaN       NaN         NaN   
4  10           NaN                      NaN       NaN       NaN         NaN   
...             ...                      ...       ...       ...         ...   
95 10           NaN                      NaN       NaN       NaN         NaN   
96 10           NaN                      NaN       NaN       NaN         NaN   
97 10           NaN                      NaN       NaN       NaN         NaN   
98 10 